In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when

# Initialize Spark session
spark = SparkSession.builder.appName("Gold Layer Processing").getOrCreate()

# Snowflake connection options
sfOptions = {
    "sfURL": "https://wsxmajg-ms55786.snowflakecomputing.com",
    "sfUser": "KAICY3108",
    "sfPassword": "Kaicy@11122021",
    "sfDatabase": "AARAMBH",
    "sfSchema": "SILVER",
    "sfWarehouse": "COMPUTE_WH"
}

# Snowflake read function
def read_from_snowflake(table_name):
    return spark.read \
        .format("snowflake") \
        .options(**sfOptions) \
        .option("dbtable", table_name) \
        .load()

# Read data from the Silver layer
silver_df = read_from_snowflake("SILVER.SILVER_CLEANED_DATA")

# Additional Data Processing (Aggregation & Business Logic)
gold_df = silver_df.withColumn("customer_risk_category", 
                when((col("credit_score") >= 700) & (col("income_category") == "High"), "Low Risk")
                .when((col("credit_score") >= 500) & (col("income_category") != "Low"), "Medium Risk")
                .otherwise("High Risk"))

# Selecting final transformed columns
gold_df = gold_df.select(
    "cust_id", "name", "age", "credit_score", "income", "income_category", "offer_type", "offer_status", "customer_risk_category"
)

# Display final DataFrame
gold_df.show()

# Snowflake write function
def write_to_snowflake(df, table_name):
    df.write \
        .format("snowflake") \
        .options(**sfOptions) \
        .option("dbtable", table_name) \
        .mode("overwrite") \
        .save()

# Switch schema to GOLD for loading final table
sfOptions["sfSchema"] = "GOLD"

# Load final processed data into Gold layer
write_to_snowflake(gold_df, "GOLD.AARAMBH_TABLE")

# Read back from Gold layer for verification
gold_verified_df = read_from_snowflake("GOLD.AARAMBH_TABLE")
gold_verified_df.show()
